## How do countries make money from trade?

- UK and US comparisons for each chart.

In [6]:
import pandas as pd
import requests
import json
import altair as alt
from ecostyles import EcoStyles
styles = EcoStyles()
styles.register_and_enable_theme(theme_name='article')
p21 = ['GBR', 'USA']


In [1]:
# series_code = 'GGXONLB_G01_GDP_PT'
series_code = 'GGXCNL_NGDP'

def get_imf_data(series_code, year:int=None):
    df_main = pd.DataFrame()
    for iso3 in p21:
        if year is not None:
            imf_api = f'https://www.imf.org/external/datamapper/api/v1/{series_code}/{iso3}?periods={year}'
        else: # fetch for all years
            imf_api = f'https://www.imf.org/external/datamapper/api/v1/{series_code}/{iso3}'

        response = requests.get(imf_api)

        data = json.loads(response.text)

        # Try and access `values` key, (if not, then IMF Fiscal Monitor has no data for this country)
        try:
            df = pd.DataFrame(data['values'][f'{series_code}'])
        except:
            if year is not None:
                print(f'No data for {iso3}, retrying for {year-1}')
                # Try for previous year
                imf_api = f'https://www.imf.org/external/datamapper/api/v1/{series_code}/{iso3}?periods={year-1}'
                response = requests.get(imf_api)
                data = json.loads(response.text)
                try:
                    df = pd.DataFrame(data['values'][f'{series_code}'])
                except:
                    print(f'No data for {iso3}')
            continue

        df = df.reset_index()
        df.columns = ['Year', 'Value']
        df['ISO3'] = iso3

        df_main = pd.concat([df_main, df], ignore_index=True)

    return df_main

# df_main = get_imf_data('GGXCNL_NGDP')

NameError: name 'pd' is not defined

In [ ]:
def get_ons_data(series_code, year:int=None):
    pass

def get_fred_data(series_code, year:int=None):
    pass


#### Chart 1. Current account balance, % of GDP.

From World Economic Outlook, April 2025.

In [95]:
df_current = get_imf_data('BCA_NGDPD', '2025')

df_current['label'] = df_current['Value'].apply(lambda x: f'{x:.1f}%')

df_current['country'] = df_current['ISO3'].map({
    'GBR': 'United Kingdom',
    'USA': 'United States'
})
df_current['label']

0    -2.8%
1    -3.1%
Name: label, dtype: object

In [96]:
df_temp = df_current.copy()
df_temp['Value'] *= -1
bars = alt.Chart(df_temp).mark_bar(
    color=styles.eco_colours['turquoise'],
    height=90
).encode(
    alt.X('Value:Q').axis(
        labelExpr="'-' + datum.label + '%'"
    ),
    alt.Y('country:N').axis(
        labelFontSize=12,
        labelPadding=6
    )
)


text = alt.Chart(df_temp).mark_text(
    align='left',
    baseline='middle',
    dx=4,
    fontSize=14
).encode(
    alt.X('Value:Q'),
    alt.Y('country:N'),
    alt.Text('label:N')
)

chart = (bars + text).properties(
    width=400,
    height=230
)

chart.display()

alt.LayerChart(...)

In [97]:
styles.save(chart, 'charts', 'C1_current_account_balance', width=400, height=230)

- Title: Chart 1: Current account balance, % of GDP.
- Source: World Economic Outlook October 2024, IMF.
- Notes: Estimated for 2025.

___

#### Chart 1. Current account balance, % of GDP, over time.

UK:
- BoP: current account balance as per cent of GDP.
- https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/aa6h/ukea

In [105]:
quarters = {
    ' Q1': '-01-01',
    ' Q2': '-04-01',
    ' Q3': '-07-01',
    ' Q4': '-10-01'
}

In [116]:
series = 'aa6h/ukea'
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}
req = requests.get(f'https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/{series}/data', headers=headers)
data = json.loads(req.text)
# Quarterly data
df_uk = pd.DataFrame(data['years'])
df_uk = df_uk[['date', 'label', 'value']]
# Replace quarters with dates
# df_uk['date'] = df_uk['date'].apply(lambda x: f'{x[:4]}{quarters[x[4:]]}')
# df_uk['date'] = df_uk['date'].map(quarters).fillna(df_uk['date'])
df_uk['value'] = df_uk['value'].astype(float)

df_uk['country'] = 'United Kingdom'
df_uk['ISO3'] = 'GBR'
df_uk['value'] /= 100
df_uk

,date,label,value,country,ISO3
0,1948,1948,0.008,United Kingdom,GBR
1,1949,1949,0.004,United Kingdom,GBR
2,1950,1950,0.028,United Kingdom,GBR
3,1951,1951,-0.022,United Kingdom,GBR
4,1952,1952,0.016,United Kingdom,GBR
...,...,...,...,...,...
72,2020,2020,-0.029,United Kingdom,GBR
73,2021,2021,-0.004,United Kingdom,GBR
74,2022,2022,-0.021,United Kingdom,GBR
75,2023,2023,-0.035,United Kingdom,GBR


In [117]:
bars_uk =alt.Chart(df_uk).mark_bar().encode(
    alt.X('date:T'),
    alt.Y('value:Q').axis(
        gridDash=alt.expr('datum.value == 0 ? [0,0] : [1,5]'),                  
        format='%'
    )
)

bars_uk.properties(
    width=430,
    height=220
)

alt.Chart(...)

#### TODO: US Current account balance, % of GDP.

<br>

---

<br>

#### Chart 2. Trade balance, % of GDP.

- Separated by goods, services, and total.

ONS (https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/):
- Trade balance: d28l/pnbp
- goods: d28j/pnbp
- services: d28k/pnbp

In [54]:
ons_series = {
    'Total': 'd28l/pnbp',
    'Goods': 'd28j/pnbp',
    'Services': 'd28k/pnbp'
}

In [162]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36'
}

df_uk = pd.DataFrame()
for series in ons_series.keys():
    req = requests.get(f'https://www.ons.gov.uk/economy/nationalaccounts/balanceofpayments/timeseries/{ons_series[series]}/data', headers=headers)

    data = json.loads(req.text)

    # Quarterly data
    df = pd.DataFrame(data['quarters'])

    df = df[['date', 'label', 'value']]

    df['country'] = 'United Kingdom'
    df['ISO3'] = 'GBR'
    df['series'] = series

    df_uk = pd.concat([df_uk, df], ignore_index=True)


# Replace quarters with dates
df_uk['date'] = df_uk['date'].apply(lambda x: f'{x[:4]}{quarters[x[4:]]}')
# df_uk['date'] = df_uk['date'].map(quarters).fillna(df_uk['date'])
df_uk['value'] = df_uk['value'].astype(float)
# Convert to percentage
df_uk['value'] /= 100
df_uk


,date,label,value,country,ISO3,series
0,1955-01-01,1955 Q1,-0.015,United Kingdom,GBR,Total
1,1955-04-01,1955 Q2,-0.014,United Kingdom,GBR,Total
2,1955-07-01,1955 Q3,-0.015,United Kingdom,GBR,Total
3,1955-10-01,1955 Q4,-0.009,United Kingdom,GBR,Total
4,1956-01-01,1956 Q1,0.001,United Kingdom,GBR,Total
...,...,...,...,...,...,...
835,2023-10-01,2023 Q4,0.066,United Kingdom,GBR,Services
836,2024-01-01,2024 Q1,0.067,United Kingdom,GBR,Services
837,2024-04-01,2024 Q2,0.070,United Kingdom,GBR,Services
838,2024-07-01,2024 Q3,0.068,United Kingdom,GBR,Services


In [170]:
# Filter to 1992 onwards (to match FRED data)
df_uk2 = df_uk[df_uk['date'] >= '1992-01-01'].copy()

# Smooth with rolling average
df_uk2['value_smooth'] = df_uk2.groupby('series')['value'].transform(lambda x: x.rolling(window=4).mean())

lines_uk = alt.Chart(df_uk2).mark_line().encode(
    alt.X('date:T'),
    alt.Y('value_smooth:Q').axis(
        gridDash=alt.expr('datum.value == 0 ? [0,0] : [1,5]'),                  
        format='%'
    ),
    alt.Color('series:N')
)

chart_uk = lines_uk.properties(
    height=200
)
chart_uk.display()


alt.Chart(...)

In [100]:
styles.save(lines_uk, 'charts', 'C2_trade_balance_uk', width=400, height=230)

<br>

FRED: Trade Balance, millions of dollars, monthly.
- Total: Trade Balance: Goods and Services, Balance of Payments Basis (BOPGSTB)
- Goods: Trade Balance: Goods, Balance of Payments Basis (BOPGTB)
- Services: Trade Balance: Services, Balance of Payments Basis - BOPSTB


Need GDP data also (only quarterly data available):
- GDP: GDP
- Real GDP: GDPC1

In [150]:
fred_series = {
    'Total': 'BOPGSTB',
    'Goods': 'BOPGTB',
    'Services': 'BOPSTB'
}

df_fred = pd.DataFrame()
for series in fred_series.keys():
    ## Monthly
    # url = f'https://eco-cors-proxy.netlify.app/proxy?url=https://api.stlouisfed.org/fred/series/observations?series_id={fred_series[series]}&api_key=838a40e4f5a37b6b4d8c9cfc4b1abaff&file_type=json&frequency=m'
    ## Sum to quarterly
    url = f'https://eco-cors-proxy.netlify.app/proxy?url=https://api.stlouisfed.org/fred/series/observations?series_id={fred_series[series]}&api_key=838a40e4f5a37b6b4d8c9cfc4b1abaff&file_type=json&frequency=q&aggregation_method=sum'

    req = requests.get(url)
    data = json.loads(req.text)
    df = pd.DataFrame(data['observations'])
    df = df[['date', 'value']]

    # If final row is '2025-01-01', remove it
    if df['date'].iloc[-1] == '2025-01-01':
        df = df.iloc[:-1]

    df['country'] = 'United States'
    df['ISO3'] = 'USA'
    df['series'] = series

    df_fred = pd.concat([df_fred, df], ignore_index=True)

df_fred

,date,value,country,ISO3,series
0,1992-01-01,-5498,United States,USA,Total
1,1992-04-01,-9852,United States,USA,Total
2,1992-07-01,-10766,United States,USA,Total
3,1992-10-01,-13091,United States,USA,Total
4,1993-01-01,-14244,United States,USA,Total
...,...,...,...,...,...
391,2023-10-01,70615,United States,USA,Services
392,2024-01-01,73813,United States,USA,Services
393,2024-04-01,72069,United States,USA,Services
394,2024-07-01,73220,United States,USA,Services


Merge with GDP data.
- GDP data is annual level (is it at quarterly rate?), in billions.
- Trade data is monthly level, in millions.

In [151]:
series = 'GDP'
url = f'https://eco-cors-proxy.netlify.app/proxy?url=https://api.stlouisfed.org/fred/series/observations?series_id={series}&api_key=838a40e4f5a37b6b4d8c9cfc4b1abaff&file_type=json&frequency=q'
req = requests.get(url)
data = json.loads(req.text)
df_gdp = pd.DataFrame(data['observations'])
df_gdp = df_gdp[['date', 'value']]
df_gdp.columns = ['date', 'GDP']

# In billions. Convert to millions so equivalent to trade data.
df_gdp = df_gdp[df_gdp['GDP'] != '.'].copy()
df_gdp['GDP'] = df_gdp['GDP'].astype(float)
df_gdp['GDP'] *= 1000
df_gdp


,date,GDP
4,1947-01-01,243164.0
5,1947-04-01,245968.0
6,1947-07-01,249585.0
7,1947-10-01,259745.0
8,1948-01-01,265742.0
...,...,...
311,2023-10-01,28296967.0
312,2024-01-01,28624069.0
313,2024-04-01,29016714.0
314,2024-07-01,29374914.0


In [152]:
df_fred2 = pd.merge(df_fred, df_gdp, on='date')

df_fred2['value'] = df_fred2['value'].astype(float)
df_fred2['GDP'] = df_fred2['GDP'].astype(float)

# Convert from annual to"actual quarter"
df_fred2['GDP_actual_qtr'] = (df_fred2['GDP'] / 4.0)
df_fred2['value_gdp'] = (df_fred2['value'] / df_fred2['GDP_actual_qtr'])



# df_fred

In [154]:
df_fred2

,date,value,country,ISO3,series,GDP,GDP_actual_qtr,value_gdp
0,1992-01-01,-5498.0,United States,USA,Total,6363102.0,1590775.50,-0.003456
1,1992-04-01,-9852.0,United States,USA,Total,6470763.0,1617690.75,-0.006090
2,1992-07-01,-10766.0,United States,USA,Total,6566641.0,1641660.25,-0.006558
3,1992-10-01,-13091.0,United States,USA,Total,6680803.0,1670200.75,-0.007838
4,1993-01-01,-14244.0,United States,USA,Total,6729459.0,1682364.75,-0.008467
...,...,...,...,...,...,...,...,...
391,2023-10-01,70615.0,United States,USA,Services,28296967.0,7074241.75,0.009982
392,2024-01-01,73813.0,United States,USA,Services,28624069.0,7156017.25,0.010315
393,2024-04-01,72069.0,United States,USA,Services,29016714.0,7254178.50,0.009935
394,2024-07-01,73220.0,United States,USA,Services,29374914.0,7343728.50,0.009970


In [155]:
df_fred.groupby('series')['date'].min()

series
Goods       1992-01-01
Services    1992-01-01
Total       1992-01-01
Name: date, dtype: object

** Transformation currently incorrect

In [169]:
lines_usa = alt.Chart(df_fred2).mark_line().encode(
    alt.X('date:T'),
    alt.Y('value_gdp:Q').axis(
        gridDash=alt.expr("datum.value == 0 ? [0, 0] : [1, 5]"),
        format='%'
    ),
    alt.Color('series:N')
)

chart_usa = lines_usa.properties(
    height=200
)

chart_usa.display()


alt.Chart(...)

In [171]:
alt.vconcat(chart_uk, chart_usa).display()

alt.VConcatChart(...)